In [1]:
# https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/home
import json
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2.0.0


In [2]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = "post"
oov_tok = "<OOV>"
training_size = 20000
padding_type = "post"

In [3]:
with open("./sarcasm.json") as f:
    datastore = json.load(f)

In [4]:
sentences = []
labels = []
urls = []

In [5]:
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [6]:
training_sentences = sentences[0:training_size]
testing_sentences =  sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [7]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss='binary_crossentropy', optimizer = 'adam', metric=['acc'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 16)            160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 30

training_padded_final = np.array(training_padded)
training_labels_final = np.array(training_labels)
testing_padded_final = np.array(testing_padded)
testing_labels_final = np.array(testing_labels)



history = model.fit(training_padded_final, 
                    training_labels_final, 
                    epochs = num_epochs, 
                    validation_data=(testing_padded_final, testing_labels_final),
                    verbose=2)

Train on 20000 samples, validate on 6709 samples
Epoch 1/30
20000/20000 - 2s - loss: 0.5728 - val_loss: 0.4013
Epoch 2/30
20000/20000 - 2s - loss: 0.3144 - val_loss: 0.3438
Epoch 3/30
20000/20000 - 2s - loss: 0.2352 - val_loss: 0.3446
Epoch 4/30
20000/20000 - 2s - loss: 0.1889 - val_loss: 0.3638
Epoch 5/30
20000/20000 - 2s - loss: 0.1572 - val_loss: 0.3925
Epoch 6/30
20000/20000 - 2s - loss: 0.1341 - val_loss: 0.4280
Epoch 7/30
20000/20000 - 2s - loss: 0.1144 - val_loss: 0.4596
Epoch 8/30
20000/20000 - 2s - loss: 0.0989 - val_loss: 0.5003
Epoch 9/30
20000/20000 - 2s - loss: 0.0869 - val_loss: 0.5524
Epoch 10/30
20000/20000 - 2s - loss: 0.0749 - val_loss: 0.6035
Epoch 11/30
20000/20000 - 2s - loss: 0.0660 - val_loss: 0.6427
Epoch 12/30
20000/20000 - 2s - loss: 0.0574 - val_loss: 0.6980
Epoch 13/30
20000/20000 - 2s - loss: 0.0498 - val_loss: 0.7725
Epoch 14/30
20000/20000 - 2s - loss: 0.0433 - val_loss: 0.8223
Epoch 15/30
20000/20000 - 2s - loss: 0.0395 - val_loss: 0.8688
Epoch 16/30
200

In [19]:
import matplotlib.pyplot as plt

def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

plot_graph(history, "acc")
plot_graph(history, "loss")

KeyError: 'acc'